In [ ]:
#Juego del gato (4x4)
#Algoritmo minimax con poda αβ
#	Humano VS Humano
#	Humano VS IA
#	IA VS IA
import numpy as np
from typing import List, Tuple, Optional
import time

class TicTacToe4x4:
    def __init__(self):
        self.board = np.zeros((4, 4), dtype=int)
        self.EMPTY = 0
        self.PLAYER_X = 1
        self.PLAYER_O = -1
        self.MAX_DEPTH = 5  # Limitamos la profundidad de búsqueda

    def print_board(self):
        """Imprime el tablero de forma amigable"""
        symbols = {0: ' ', 1: 'X', -1: 'O'}
        print('\n')
        print('     0   1   2   3')
        print('   ┌───┬───┬───┬───┐')
        for i in range(4):
            print(f' {i} │', end=' ')
            for j in range(4):
                print(f"{symbols[self.board[i][j]]} │", end=' ')
            print()
            if i < 3:
                print('   ├───┼───┼───┼───┤')
        print('   └───┴───┴───┴───┘')

    def make_move(self, row: int, col: int, player: int) -> bool:
        """Realiza un movimiento en el tablero"""
        if self.is_valid_move(row, col):
            self.board[row][col] = player
            return True
        return False

    def is_valid_move(self, row: int, col: int) -> bool:
        """Verifica si un movimiento es válido"""
        return 0 <= row < 4 and 0 <= col < 4 and self.board[row][col] == self.EMPTY

    def get_valid_moves(self) -> List[Tuple[int, int]]:
        """Retorna lista de movimientos válidos"""
        moves = []
        for i in range(4):
            for j in range(4):
                if self.board[i][j] == self.EMPTY:
                    moves.append((i, j))
        return moves

    def check_winner(self) -> Optional[int]:
        """Verifica si hay un ganador"""
        # Verificar filas y columnas
        for i in range(4):
            if abs(sum(self.board[i,:])) == 4:
                return self.board[i,0]
            if abs(sum(self.board[:,i])) == 4:
                return self.board[0,i]

        # Verificar diagonales
        diag = sum([self.board[i,i] for i in range(4)])
        anti_diag = sum([self.board[i,3-i] for i in range(4)])

        if abs(diag) == 4:
            return self.board[0,0]
        if abs(anti_diag) == 4:
            return self.board[0,3]

        return None

    def is_full(self) -> bool:
        """Verifica si el tablero está lleno"""
        return self.EMPTY not in self.board

    def minimax_alpha_beta(self, depth: int, alpha: float, beta: float, maximizing: bool) -> Tuple[int, Optional[Tuple[int, int]]]:
        """Implementación de Minimax con poda alfa-beta"""
        winner = self.check_winner()

        if winner is not None:
            return winner * 100, None

        if self.is_full() or depth >= self.MAX_DEPTH:
            return self.evaluate_board(), None

        valid_moves = self.get_valid_moves()
        best_move = None

        if maximizing:
            max_eval = float('-inf')
            for move in valid_moves:
                self.board[move[0]][move[1]] = self.PLAYER_X
                eval, _ = self.minimax_alpha_beta(depth + 1, alpha, beta, False)
                self.board[move[0]][move[1]] = self.EMPTY

                if eval > max_eval:
                    max_eval = eval
                    best_move = move

                alpha = max(alpha, eval)
                if beta <= alpha:
                    break

            return max_eval, best_move
        else:
            min_eval = float('inf')
            for move in valid_moves:
                self.board[move[0]][move[1]] = self.PLAYER_O
                eval, _ = self.minimax_alpha_beta(depth + 1, alpha, beta, True)
                self.board[move[0]][move[1]] = self.EMPTY

                if eval < min_eval:
                    min_eval = eval
                    best_move = move

                beta = min(beta, eval)
                if beta <= alpha:
                    break

            return min_eval, best_move

    def evaluate_board(self) -> int:
        """Evalúa el estado actual del tablero"""
        score = 0

        # Evaluar filas y columnas
        for i in range(4):
            row_sum = sum(self.board[i,:])
            col_sum = sum(self.board[:,i])

            if abs(row_sum) == 3:
                score += 10 if row_sum > 0 else -10
            if abs(col_sum) == 3:
                score += 10 if col_sum > 0 else -10

        # Evaluar diagonales
        diag = sum([self.board[i,i] for i in range(4)])
        anti_diag = sum([self.board[i,3-i] for i in range(4)])

        if abs(diag) == 3:
            score += 10 if diag > 0 else -10
        if abs(anti_diag) == 3:
            score += 10 if anti_diag > 0 else -10

        return score

    def get_ai_move(self, player: int) -> Tuple[int, int]:
        """Obtiene el mejor movimiento para la IA"""
        _, move = self.minimax_alpha_beta(0, float('-inf'), float('inf'), player == self.PLAYER_X)
        return move

    def play_game(self, mode: str):
        """Ejecuta el juego en el modo especificado"""
        current_player = self.PLAYER_X
        game_over = False

        while not game_over:
            self.print_board()

            if mode == "human-human" or \
               (mode == "human-ai" and current_player == self.PLAYER_X):
                # Turno humano
                while True:
                    try:
                        print(f"Turno del jugador {'X' if current_player == self.PLAYER_X else 'O'}")
                        row = int(input("Ingresa la fila (0-3): "))
                        col = int(input("Ingresa la columna (0-3): "))
                        if self.make_move(row, col, current_player):
                            break
                        else:
                            print("Movimiento inválido, intenta de nuevo")
                    except ValueError:
                        print("Por favor ingresa números válidos")
            else:
                # Turno IA
                print(f"\nTurno de la IA {'X' if current_player == self.PLAYER_X else 'O'}")
                time.sleep(1)  # Pequeña pausa para mejor visualización
                move = self.get_ai_move(current_player)
                self.make_move(move[0], move[1], current_player)

            # Verificar si hay ganador
            winner = self.check_winner()
            if winner is not None:
                self.print_board()
                print(f"\n¡Ganador: {'X' if winner == self.PLAYER_X else 'O'}!")
                game_over = True
            elif self.is_full():
                self.print_board()
                print("\n¡Empate!")
                game_over = True

            current_player = self.PLAYER_O if current_player == self.PLAYER_X else self.PLAYER_X

def main():
    while True:
        print("\nBienvenido al Juego del Gato 4x4")
        print("1. Humano vs Humano")
        print("2. Humano vs IA")
        print("3. IA vs IA")
        print("4. Salir")

        choice = input("\nSelecciona una opción (1-4): ")

        if choice == "4":
            break

        game = TicTacToe4x4()

        if choice == "1":
            game.play_game("human-human")
        elif choice == "2":
            game.play_game("human-ai")
        elif choice == "3":
            game.play_game("ai-ai")
        else:
            print("Opción inválida")

if __name__ == "__main__":
    main()


Bienvenido al Juego del Gato 4x4
1. Humano vs Humano
2. Humano vs IA
3. IA vs IA
4. Salir

Selecciona una opción (1-4): 3


     0   1   2   3
   ┌───┬───┬───┬───┐
 0 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 1 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 2 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 3 │   │   │   │   │ 
   └───┴───┴───┴───┘

Turno de la IA X


     0   1   2   3
   ┌───┬───┬───┬───┐
 0 │ X │   │   │   │ 
   ├───┼───┼───┼───┤
 1 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 2 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 3 │   │   │   │   │ 
   └───┴───┴───┴───┘

Turno de la IA O


     0   1   2   3
   ┌───┬───┬───┬───┐
 0 │ X │ O │   │   │ 
   ├───┼───┼───┼───┤
 1 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 2 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 3 │   │   │   │   │ 
   └───┴───┴───┴───┘

Turno de la IA X


     0   1   2   3
   ┌───┬───┬───┬───┐
 0 │ X │ O │   │   │ 
   ├───┼───┼───┼───┤
 1 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 2 │   │   │   │   │ 
   ├───┼───┼───┼───┤
 3 │ X │  